notebook [link](https://www.kaggle.com/code/sayamkumar/twitter-sentiment-analysis)

In [2]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential

In [3]:
df = pd.read_csv('..\datasets\Reddit_Data.csv')
df.head()

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


### Data Preprocessing

Remove null cells

In [4]:
df.dropna(inplace=True)

Remove punctuation

In [5]:
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
def remove_punc(text):
    no_punc_text = [char for char in text if char not in string.punctuation]
    no_punc_text = ''.join(no_punc_text)
    return ' '.join([word for word in no_punc_text.split() if word.lower() not in stopwords.words('english') and not word.isdigit()])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
df.clean_comment = df.clean_comment.apply(remove_punc)

Stemmerization, Lemmatization, Spell_correction

In [7]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')
def stem_words(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])

In [8]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

In [9]:
from symspellpy import SymSpell, Verbosity
symspell = SymSpell()
def spelling_correction(text):
    correct_spellings = []
    for token in text.split():
        x = symspell.lookup(token,Verbosity.CLOSEST,max_edit_distance=2,include_unknown=True)[0].__str__()
        y = x.split(',')[0]
        correct_spellings.append(y)
    return ' '.join(correct_spellings)

In [10]:
df.clean_comment = df.clean_comment.apply(stem_words)
df.clean_comment = df.clean_comment.apply(lemmatize_words)
df.clean_comment = df.clean_comment.apply(spelling_correction)

In [11]:
df.clean_comment

0        famili mormon never tri explain still stare pu...
1        buddhism much lot compat christian especi cons...
2        serious say thing first get complex explain no...
3        learn want teach differ focus goal wrap paper ...
4        benefit may want read live buddha live christ ...
                               ...                        
37244                                                jesus
37245    kya bhai pure saal chutiya banaya modi aur jab...
37246                   downvot karna tha par upvot hogaya
37247                                            haha nice
37248                               facebook work bjp cell
Name: clean_comment, Length: 37149, dtype: object

In [14]:
df.loc[df.clean_comment.str.isspace()]

,clean_comment,category
